In [1]:
%load_ext autoreload
%autoreload 2

In [107]:
from pathlib import Path

data_dir = Path("sample")
all_files = ["solar-mini.txt", "finetune.txt", "proofreading.txt"]
target_file = all_files[2]

with open(data_dir / target_file, "r") as f:
    blog = f.read()

In [108]:
from langchain_upstage import ChatUpstage as Chat
from langchain_community.tools import DuckDuckGoSearchResults

MODEL_NAME = "solar-pro"
llm = Chat(model=MODEL_NAME)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) Gecko/20100101 Firefox/124.0"
}
ddg_search = DuckDuckGoSearchResults(headers=headers)
confidence_threshold = 0.7

In [109]:
from fc import extracted_claimed_facts
max_num_try = 10

for attempt in range(max_num_try):
    try:
        claimed_facts = extracted_claimed_facts(blog)
        print(f"Extracted {len(claimed_facts)} claimed facts completed.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        if attempt < max_num_try - 1:
            print(f"Retrying... (Attempt {attempt + 2}/{max_num_try})")
        else:
            print("Max retries reached. Extraction failed.")

Extracted 13 claimed facts completed.


In [110]:
from search import search_context

for attempt in range(100):
    try:
        context = search_context(blog, claimed_facts, ddg_search)
        print("Search context completed.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        if attempt < max_num_try - 1:
            print(f"Retrying... (Attempt {attempt + 2}/{max_num_try})")
        else:
            print("Max retries reached. Search context failed.")

An error occurred: https://links.duckduckgo.com/d.js?q=Upstage+Solar+LLM+customization+Solar-Mini+fine-tuning+Predibase+proofreading+media+company+large+LLMs+shortcomings+Solar-Proofread+accuracy+cost+effectiveness+automation+enterprise+AI+South+Korea+AI+company+Samsung+AIA+Hyundai+GPT-4+GPT-4o+mini+OpenAI+models+article+proofreading+reduced+operational+costs&kl=wt-wt&l=wt-wt&p=&s=0&df=y&vqd=4-40299537339027023641020061961472942235&bing_market=wt-WT&ex=-1 202 Ratelimit
Retrying... (Attempt 2/10)
Search context completed.


In [111]:
from fc import build_kg

for attempt in range(max_num_try):
    try:
        kg = build_kg(claimed_facts, context, llm)
        print("Knowledge graph built.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        if attempt < max_num_try - 1:
            print(f"Retrying... (Attempt {attempt + 2}/{max_num_try})")
        else:
            print("Max retries reached. Knowledge graph building failed.")

Knowledge graph built.


In [112]:
from fc import verify_facts

for attempt in range(max_num_try):
    try:
        verified_facts = verify_facts(claimed_facts, context, kg, confidence_threshold, llm)
        print("Facts verification completed.")
        break
    except Exception as e:
        print(f"An error occurred: {e}")
        if attempt < max_num_try - 1:
            print(f"Retrying... (Attempt {attempt + 2}/{max_num_try})")
        else:
            print("Max retries reached. Facts verification failed.")

Facts verification completed.


In [113]:
verified_facts

{'0': {'claimed': "Upstage is South Korea's most successful AI company, providing state-of-the-art enterprise AI and LLM solutions to thousands of customers like Samsung, AIA, Hyundai, and many more.",
  'Rating': 'HALF TRUE',
  'confidence': '0.70',
  'explanation': "While Upstage is a successful AI company providing LLM solutions to notable clients like Samsung, the knowledge graph does not provide information comparing Upstage's success to other AI companies in South Korea, making the claim 'most successful' only partially true."},
 '1': {'claimed': 'Having raised over $200 million since their founding in 2020, they are by far the most-funded tech company in South Korean history.',
  'Rating': 'FALSE',
  'confidence': '1.0',
  'explanation': "The knowledge graph does not provide any information about Upstage's funding history or comparison to other South Korean tech companies, making the claim inaccurate."},
 '2': {'claimed': 'They have customized their flagship LLM–Solar–to be supe

In [ ]:
# read 

In [114]:
# save all the intermediate results
import json

output_dir = (Path("output") / target_file).with_suffix("")
output_dir.mkdir(parents=True, exist_ok=True)

with open(output_dir / "claimed_facts.json", "w") as f:
    json.dump(claimed_facts, f, indent=4)

with open(output_dir / "context.json", "w") as f:
    json.dump(context.split('snippet: ')[1:], f, indent=4)

with open(output_dir / "kg.json", "w") as f:
    json.dump(kg, f, indent=4)

with open(output_dir / "verified_facts.json", "w") as f:
    json.dump(verified_facts, f, indent=4)


In [132]:
# read all the files

import json
from pathlib import Path

output_dir = Path("output")
all_files = ["solar-mini", "finetune", "proofreading"]
RATING_ORDER = ["PANTS ON FIRE", "FALSE", "MOSTLY FALSE", "HALF TRUE", "MOSTLY TRUE", "TRUE"]

for blog in all_files:
    for file in output_dir.glob(f"{blog}/verified_facts.json"):
        with open(file, "r") as f:
            data = json.load(f)
            data = sorted(data.items(), key=lambda x: RATING_ORDER.index(x[1]["Rating"]))
            data = [x[1] for x in data]
            # save the data
            with open(file, "w") as f:
                json.dump(data, f, indent=4)



In [135]:
for blog in all_files:
    for file in output_dir.glob(f"{blog}/claimed_facts.json"):
        with open(file, "r") as f:
            data = json.load(f)
            # sort the number of tags
            data = sorted(data, key=lambda x: len(x["tags"]), reverse=True)
            # save the data
            with open(file, "w") as f:
                json.dump(data, f, indent=4)


In [120]:
data

{'0': {'claimed': "Upstage is South Korea's most successful AI company, providing state-of-the-art enterprise AI and LLM solutions to thousands of customers like Samsung, AIA, Hyundai, and many more.",
  'Rating': 'HALF TRUE',
  'confidence': '0.70',
  'explanation': "While Upstage is a successful AI company providing LLM solutions to notable clients like Samsung, the knowledge graph does not provide information comparing Upstage's success to other AI companies in South Korea, making the claim 'most successful' only partially true."},
 '1': {'claimed': 'Having raised over $200 million since their founding in 2020, they are by far the most-funded tech company in South Korean history.',
  'Rating': 'FALSE',
  'confidence': '1.0',
  'explanation': "The knowledge graph does not provide any information about Upstage's funding history or comparison to other South Korean tech companies, making the claim inaccurate."},
 '2': {'claimed': 'They have customized their flagship LLM–Solar–to be supe